In [7]:
import time
start_time = time.time()
import pandas as pd
import numpy as np
import gc
from tqdm import tqdm

In [8]:
pd.set_option('display.max_rows', 99)
pd.set_option('display.max_columns', 50)

import warnings

warnings.filterwarnings('ignore')


# Data path
data_path = 'data'
submission_path = ''

In [9]:
def downcast_dtypes(df):

    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols =   [c for c in df if df[c].dtype in ["int64", "int32"]]

    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols]   = df[int_cols].astype(np.int16)

    return df


In [10]:
# 0. Load data ----------------------------------------------------------------

print('%0.2f min: Start loading result'%((time.time() - start_time)/60))

result = pd.read_csv('%s/result/ver6_lr_stacking.csv' % data_path)
result.to_csv('ver6_lr_stacking.csv', index = False)

print('%0.2f min: Finish loading result'%((time.time() - start_time)/60))


0.05 min: Start loading result


FileNotFoundError: File b'data/result/ver6_lr_stacking.csv' does not exist

In [12]:
sale_train = pd.read_csv('%s/sales_train_v2.csv' % data_path)
test  = pd.read_csv('%s/test.csv' % data_path)

sale_train[sale_train['item_id'] == 11373][['item_price']].sort_values(['item_price'])
sale_train[sale_train['item_id'] == 11365].sort_values(['item_price'])

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
1651714,16.05.2014,16,12,11365,124.000000,5.0
2805487,21.08.2015,31,12,11365,170.000000,2.0
1330776,13.01.2014,12,12,11365,180.000000,3.0
1398688,25.02.2014,13,12,11365,194.000000,5.0
661581,05.07.2013,6,12,11365,230.000000,4.0
2731490,30.07.2015,30,12,11365,230.000000,1.0
1398683,06.02.2014,13,12,11365,246.666667,9.0
2626123,21.05.2015,28,12,11365,248.333333,6.0
2608316,16.04.2015,27,12,11365,249.333333,15.0
1058265,29.11.2013,10,12,11365,260.867925,53.0


In [13]:
 print('%0.2f min: Start loading data'%((time.time() - start_time)/60))
# Correct sale_train values

sale_train['item_price'][2909818] = np.nan

sale_train['item_cnt_day'][2909818] = np.nan

sale_train['item_price'][2909818] = sale_train[(sale_train['shop_id'] ==12) & (sale_train['item_id'] == 11373) & (sale_train['date_block_num'] == 33)]['item_price'].median()

sale_train['item_cnt_day'][2909818] = round(sale_train[(sale_train['shop_id'] ==12) & (sale_train['item_id'] == 11373) & (sale_train['date_block_num'] == 33)]['item_cnt_day'].median())

sale_train['item_price'][885138] = np.nan

sale_train['item_price'][885138] = sale_train[(sale_train['item_id'] == 11365) & (sale_train['shop_id'] ==12) & (sale_train['date_block_num'] == 8)]['item_price'].median()

test_nrow = test.shape[0]

sale_train = sale_train.merge(test[['shop_id']].drop_duplicates(), how = 'inner')

sale_train['date'] = pd.to_datetime(sale_train['date'], format = '%d.%m.%Y')

print('%0.2f min: Finish loading data'%((time.time() - start_time)/60))

3.08 min: Start loading data
3.22 min: Finish loading data


In [14]:
 # 1. Aggregate data ----------------------------------------------------------------

 from itertools import product

# For every month we create a grid from all shops/items combinations from that month

 grid = []

 for block_num in sale_train['date_block_num'].unique():

     cur_shops = sale_train[sale_train['date_block_num']==block_num]['shop_id'].unique()

     cur_items = sale_train[sale_train['date_block_num']==block_num]['item_id'].unique()

     grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))

 #turn the grid into pandas dataframe

 index_cols = ['shop_id', 'item_id', 'date_block_num']

 grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

 print('%0.2f min: Finish creating the grid'%((time.time() - start_time)/60))

3.34 min: Finish creating the grid


In [15]:
index_cols = ['shop_id', 'item_id', 'date_block_num']

sale_train['item_cnt_day'] = sale_train['item_cnt_day'].clip(0,20)

gb_cnt = sale_train.groupby(index_cols)['item_cnt_day'].agg(['sum']).reset_index().rename(columns = {'sum': 'item_cnt_month'})

gb_cnt['item_cnt_month'] = gb_cnt['item_cnt_month'].clip(0,20).astype(np.int)


#join aggregated data to the grid

train = pd.merge(grid,gb_cnt,how='left',on=index_cols).fillna(0)

train['item_cnt_month'] = train['item_cnt_month'].astype(int)

train = downcast_dtypes(train)


#sort the data

train.sort_values(['date_block_num','shop_id','item_id'],inplace=True)

print('%0.2f min: Finish joining gb_cnt'%((time.time() - start_time)/60))

3.55 min: Finish joining gb_cnt


In [16]:
#
 # # Sanity check
#
 # sale_train['item_cnt_day'].sum()
#
 # train['item_cnt_month'].sum()
#
 # gb_cnt['item_cnt_month'].sum()
#
#
#
#
#
item = pd.read_csv('%s/items.csv' % data_path)
#
train = train.merge(item[['item_id', 'item_category_id']], on = ['item_id'], how = 'left')
#
test = test.merge(item[['item_id', 'item_category_id']], on = ['item_id'], how = 'left')
#
print('%0.2f min: Finish adding item_category_id'%((time.time() - start_time)/60))
#
#
#
#
#
item_cat = pd.read_csv('%s/item_categories.csv' % data_path)
#
l_cat = list(item_cat.item_category_name)
#
for ind in range(0,1):
#
    l_cat[ind] = 'PC Headsets / Headphones'
#
for ind in range(1,8):
#
    l_cat[ind] = 'Access'
#
l_cat[8] = 'Tickets (figure)'
#
l_cat[9] = 'Delivery of goods'
#
for ind in range(10,18):
#
    l_cat[ind] = 'Consoles'
#
for ind in range(18,25):
#
    l_cat[ind] = 'Consoles Games'
#
    l_cat[25] = 'Accessories for games'
#
for ind in range(26,28):
#
    l_cat[ind] = 'phone games'
#
for ind in range(28,32):
#
    l_cat[ind] = 'CD games'
#
for ind in range(32,37):
#
    l_cat[ind] = 'Card'
#
for ind in range(37,43):
#
    l_cat[ind] = 'Movie'
#
for ind in range(43,55):
#
    l_cat[ind] = 'Books'
#
for ind in range(55,61):
#
    l_cat[ind] = 'Music'
#
for ind in range(61,73):
#
    l_cat[ind] = 'Gifts'
#
for ind in range(73,79):
#
    l_cat[ind] = 'Soft'
#
for ind in range(79,81):
#
    l_cat[ind] = 'Office'
#
for ind in range(81,83):
#
    l_cat[ind] = 'Clean'
#
    l_cat[83] = 'Elements of a food'
#
#
#

3.71 min: Finish adding item_category_id


In [17]:
 from sklearn import preprocessing
#
 lb = preprocessing.LabelEncoder()
#
 item_cat['item_cat_id_fix'] = lb.fit_transform(l_cat)
#
#
#
 train = train.merge(item_cat[['item_cat_id_fix', 'item_category_id']], on = ['item_category_id'], how = 'left')
#
 test = test.merge(item_cat[['item_cat_id_fix', 'item_category_id']], on = ['item_category_id'], how = 'left')
#
#
#
 del item, item_cat, grid, gb_cnt
#
 gc.collect()
#
 print('%0.2f min: Finish adding item_cat_id_fix'%((time.time() - start_time)/60))
#
#
#
#
#
#
#
 # 2. Add item/shop pair mean-encodings -----------------------------------------
#
 # For Trainset
#
 print('%0.2f min: Start adding mean-encoding for item_cnt_month'%((time.time() - start_time)/60))
#
 Target = 'item_cnt_month'
#
 global_mean =  train[Target].mean()
#
 y_tr = train[Target].values
#
#
#
 mean_encoded_col = ['shop_id', 'item_id', 'item_category_id', 'item_cat_id_fix']
#
 for col in tqdm(mean_encoded_col):
#
#
#
     col_tr = train[[col] + [Target]]
#
     corrcoefs = pd.DataFrame(columns = ['Cor'])
#

4.72 min: Finish adding item_cat_id_fix
4.72 min: Start adding mean-encoding for item_cnt_month


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00,  9.84it/s]


In [18]:
# 3.1.1 Mean encodings - KFold scheme
#
from sklearn.model_selection import KFold
#
     kf = KFold(n_splits = 5, shuffle = False, random_state = SEED)
#
#
#
     col_tr[col + '_cnt_month_mean_Kfold'] = global_mean
#
     for tr_ind, val_ind in kf.split(col_tr):
#
         X_tr, X_val = col_tr.iloc[tr_ind], col_tr.iloc[val_ind]
#
         means = X_val[col].map(X_tr.groupby(col)[Target].mean())
#
         X_val[col + '_cnt_month_mean_Kfold'] = means
#
         col_tr.iloc[val_ind] = X_val
#
         # X_val.head()
#
     col_tr.fillna(global_mean, inplace = True)
#
     corrcoefs.loc[col + '_cnt_month_mean_Kfold'] = np.corrcoef(y_tr, col_tr[col + '_cnt_month_mean_Kfold'])[0][1]
#
#
#

NameError: name 'SEED' is not defined